In [1]:

# Import necessary libraries
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dropout, Attention, Concatenate
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Read the dataset (English-to-French translation)
with open("fra.txt", encoding='utf-8') as file:
    lines = file.read().strip().split('\n')

# Use first 10,000 sentences for manageable computation
lines = lines[:10000]



In [2]:
# Preprocess: Clean and split into English and French sentences
english_sentences = []
french_sentences = []
for line in lines:
    eng, fr, _ = line.split('\t')  # Ignore third column (attribution)
    # Clean text: lowercase, remove punctuation, add start/end tokens
    eng = re.sub(r'[^\w\s]', '', eng.lower()).strip()
    fr = re.sub(r'[^\w\s]', '', fr.lower()).strip()
    english_sentences.append('<start> ' + eng + ' <end>')
    french_sentences.append('<start> ' + fr + ' <end>')

# Tokenize English sentences
eng_tokenizer = Tokenizer(filters='')
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)

# Tokenize French sentences
fr_tokenizer = Tokenizer(filters='')
fr_tokenizer.fit_on_texts(french_sentences)
fr_sequences = fr_tokenizer.texts_to_sequences(french_sentences)

# Pad sequences to ensure uniform length
max_eng_len = max(len(seq) for seq in eng_sequences)
max_fr_len = max(len(seq) for seq in fr_sequences)
eng_padded = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
fr_padded = pad_sequences(fr_sequences, maxlen=max_fr_len, padding='post')

# Vocabulary sizes
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fr_vocab_size = len(fr_tokenizer.word_index) + 1

# Split data into training and validation sets (80-20 split)
eng_train, eng_val, fr_train, fr_val = train_test_split(
    eng_padded, fr_padded, test_size=0.2, random_state=42
)



In [3]:
# Prepare decoder inputs and targets
decoder_input_train = fr_train[:, :-1]  # Exclude last token
decoder_target_train = fr_train[:, 1:]  # Exclude first token
decoder_input_val = fr_val[:, :-1]
decoder_target_val = fr_val[:, 1:]

# Print shapes to verify
print(f"English padded shape: {eng_padded.shape}")
print(f"French padded shape: {fr_padded.shape}")
print(f"Training data: {eng_train.shape}, Validation data: {eng_val.shape}")



English padded shape: (10000, 6)
French padded shape: (10000, 12)
Training data: (8000, 6), Validation data: (2000, 6)


In [4]:
# Task 4: Build Encoder and Decoder using LSTM (Keras)
#
# We define an encoder-decoder model with Embedding and LSTM layers, compile it, and train for 15 epochs, printing the training
# loss after each epoch.

# Model parameters
embedding_size = 256
lstm_units = 512
batch_size = 64
epochs = 30
dropout_rate = 0.2

# Build encoder
encoder_input = Input(shape=(None,))
enc_embedding = Embedding(eng_vocab_size, embedding_size)(encoder_input)
enc_lstm = LSTM(lstm_units, return_state=True, dropout=dropout_rate)
enc_output, state_h, state_c = enc_lstm(enc_embedding)
encoder_states = [state_h, state_c]

# Build decoder
decoder_input = Input(shape=(None,))
dec_embedding = Embedding(fr_vocab_size, embedding_size)(decoder_input)
dec_lstm = LSTM(lstm_units, return_sequences=True, return_state=True, dropout=dropout_rate)
dec_lstm_output, _, _ = dec_lstm(dec_embedding, initial_state=encoder_states)
dec_output = Dense(fr_vocab_size, activation='softmax')(dec_lstm_output)

# Create training model
model = Model([encoder_input, decoder_input], dec_output)

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
print("Model summary:")
model.summary()



Model summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, None, 256)         │         518,144 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, None, 256)         │       1,117,952 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 512), (None,      │       1,574,912 │ embedding[0][0]            │
│                               │ 512), (None, 512)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, None, 512),       │       1,574,912 │ embedding_1[0][0],         │
│                               │ (None, 512), (None, 512)] │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 4367)        │       2,240,271 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 7,026,191 (26.80 MB)

 Trainable params: 7,026,191 (26.80 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train the model and store history
print("Starting training...")
history = model.fit(
    [eng_train, decoder_input_train],
    np.expand_dims(decoder_target_train, -1),
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([eng_val, decoder_input_val], np.expand_dims(decoder_target_val, -1)),
    verbose=0
)

# Print training and validation loss and accuracy for each epoch
for epoch in range(epochs):
    print(f"Epoch {epoch+1} - Train Loss: {history.history['loss'][epoch]:.4f}, "
          f"Train Accuracy: {history.history['accuracy'][epoch]:.4f}, "
          f"Val Loss: {history.history['val_loss'][epoch]:.4f}, "
          f"Val Accuracy: {history.history['val_accuracy'][epoch]:.4f}")



Starting training...
Epoch 1 - Train Loss: 2.3502, Train Accuracy: 0.7079, Val Loss: 1.7879, Val Accuracy: 0.7533
Epoch 2 - Train Loss: 1.6755, Train Accuracy: 0.7506, Val Loss: 1.6436, Val Accuracy: 0.7624
Epoch 3 - Train Loss: 1.5039, Train Accuracy: 0.7654, Val Loss: 1.5233, Val Accuracy: 0.7746
Epoch 4 - Train Loss: 1.3531, Train Accuracy: 0.7812, Val Loss: 1.4300, Val Accuracy: 0.7851
Epoch 5 - Train Loss: 1.2206, Train Accuracy: 0.7920, Val Loss: 1.3531, Val Accuracy: 0.7956
Epoch 6 - Train Loss: 1.0998, Train Accuracy: 0.8043, Val Loss: 1.2946, Val Accuracy: 0.8046
Epoch 7 - Train Loss: 0.9966, Train Accuracy: 0.8124, Val Loss: 1.2472, Val Accuracy: 0.8135
Epoch 8 - Train Loss: 0.9065, Train Accuracy: 0.8217, Val Loss: 1.2145, Val Accuracy: 0.8196
Epoch 9 - Train Loss: 0.8210, Train Accuracy: 0.8319, Val Loss: 1.1856, Val Accuracy: 0.8223
Epoch 10 - Train Loss: 0.7472, Train Accuracy: 0.8404, Val Loss: 1.1592, Val Accuracy: 0.8271
Epoch 11 - Train Loss: 0.6805, Train Accuracy: 0

In [6]:
# Task 5: Inference and Evaluation
#


# Define inference models
# Encoder model
encoder_model = Model(encoder_input, encoder_states)

# Decoder model
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_lstm_output, state_h, state_c = dec_lstm(dec_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
dec_output = Dense(fr_vocab_size, activation='softmax')(dec_lstm_output)
decoder_model = Model([decoder_input] + decoder_states_inputs, [dec_output] + decoder_states)



In [7]:
# Function to translate a single sentence
def translate_sentence(input_sentence):
    # Preprocess input
    input_sentence = '<start> ' + re.sub(r'[^\w\s]', '', input_sentence.lower()).strip() + ' <end>'
    input_seq = eng_tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_eng_len, padding='post')

    # Get encoder states
    states = encoder_model.predict(input_seq, verbose=0)

    # Initialize target sequence with <start> token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_tokenizer.word_index['<start>']

    # Generate output sequence
    output_tokens = []
    while True:
        output_tokens_probs, h, c = decoder_model.predict([target_seq] + states, verbose=0)
        predicted_token = np.argmax(output_tokens_probs[0, -1, :])
        if predicted_token == fr_tokenizer.word_index['<end>'] or len(output_tokens) > max_fr_len:
            break
        output_tokens.append(predicted_token)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = predicted_token
        states = [h, c]

    # Convert tokens to words
    output_sentence = ' '.join([fr_tokenizer.index_word.get(token, '') for token in output_tokens])
    return output_sentence

# Test on 5 sentences
test_sentences = [
    "hello how are you",
    "I am happy",
    "what is your name",
    "good morning",
    "I love to read"
]

print("\nTranslating test sentences:\n")
for sentence in test_sentences:
    translation = translate_sentence(sentence)
    print(f"English: {sentence}")
    print(f"French: {translation}\n")




Translating test sentences:

English: hello how are you
French: dépêché dépêché furieux apprends apprends servante sauvées sauvées sauvées sauvées sauvées sauvées sauvées

English: I am happy
French: dépasse chambre chambre embrassezmoi embrassezmoi matures matures montre encre montre matures montre matures

English: what is your name
French: cuisinier givrées givrées vérifierons sennuient sennuient sauvées servante servante reposetoi givrées givrées givrées

English: good morning
French: jinterdis démissionne malédiction actuellement apprends sauvées sauvées sauvées sauvées sauvées givrées écoutons sauvées

English: I love to read
French: trompe su su tombées tombées tombées tombées abattu froncé terre froncé écoutons froncé



In [8]:
#
# Task 6: Add Basic Attention Mechanism (Bonus)


# Build encoder-decoder model with attention
# Encoder
encoder_input_attn = Input(shape=(None,))
enc_embedding_attn = Embedding(eng_vocab_size, embedding_size)(encoder_input_attn)
enc_lstm_attn = LSTM(lstm_units, return_sequences=True, return_state=True, dropout=dropout_rate)
enc_output_attn, state_h_attn, state_c_attn = enc_lstm_attn(enc_embedding_attn)
encoder_states_attn = [state_h_attn, state_c_attn]

# Decoder with attention
decoder_input_attn = Input(shape=(None,))
dec_embedding_attn = Embedding(fr_vocab_size, embedding_size)(decoder_input_attn)
dec_lstm_attn = LSTM(lstm_units, return_sequences=True, return_state=True, dropout=dropout_rate)
dec_lstm_output_attn, _, _ = dec_lstm_attn(dec_embedding_attn, initial_state=encoder_states_attn)
attention = Attention()
attn_output = attention([dec_lstm_output_attn, enc_output_attn])
dec_combined = Concatenate()([dec_lstm_output_attn, attn_output])
dec_output_attn = Dense(fr_vocab_size, activation='softmax')(dec_combined)

# Create attention model
model_attn = Model([encoder_input_attn, decoder_input_attn], dec_output_attn)
model_attn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train attention model
print("Starting training with attention...")
history_attn = model_attn.fit(
    [eng_train, decoder_input_train],
    np.expand_dims(decoder_target_train, -1),
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([eng_val, decoder_input_val], np.expand_dims(decoder_target_val, -1)),
    verbose=0
)

# Print training and validation loss for attention model
for epoch in range(epochs):
    print(f"Attention Model - Epoch {epoch+1} - Train Loss: {history_attn.history['loss'][epoch]:.4f}, "
          f"Train Accuracy: {history_attn.history['accuracy'][epoch]:.4f}, "
          f"Val Loss: {history_attn.history['val_loss'][epoch]:.4f}, "
          f"Val Accuracy: {history_attn.history['val_accuracy'][epoch]:.4f}")




Starting training with attention...
Attention Model - Epoch 1 - Train Loss: 2.3286, Train Accuracy: 0.7116, Val Loss: 1.7482, Val Accuracy: 0.7530
Attention Model - Epoch 2 - Train Loss: 1.6153, Train Accuracy: 0.7555, Val Loss: 1.5846, Val Accuracy: 0.7665
Attention Model - Epoch 3 - Train Loss: 1.4195, Train Accuracy: 0.7754, Val Loss: 1.4377, Val Accuracy: 0.7835
Attention Model - Epoch 4 - Train Loss: 1.2407, Train Accuracy: 0.7934, Val Loss: 1.3322, Val Accuracy: 0.8024
Attention Model - Epoch 5 - Train Loss: 1.0828, Train Accuracy: 0.8095, Val Loss: 1.2667, Val Accuracy: 0.8125
Attention Model - Epoch 6 - Train Loss: 0.9417, Train Accuracy: 0.8240, Val Loss: 1.2147, Val Accuracy: 0.8219
Attention Model - Epoch 7 - Train Loss: 0.8218, Train Accuracy: 0.8341, Val Loss: 1.1848, Val Accuracy: 0.8278
Attention Model - Epoch 8 - Train Loss: 0.7156, Train Accuracy: 0.8462, Val Loss: 1.1682, Val Accuracy: 0.8300
Attention Model - Epoch 9 - Train Loss: 0.6259, Train Accuracy: 0.8563, Val 

In [ ]:
# Inference models for attention
encoder_model_attn = Model(encoder_input_attn, [enc_output_attn] + encoder_states_attn)
decoder_state_input_h_attn = Input(shape=(lstm_units,))
decoder_state_input_c_attn = Input(shape=(lstm_units,))
decoder_states_inputs_attn = [decoder_state_input_h_attn, decoder_state_input_c_attn]
enc_output_input_attn = Input(shape=(None, lstm_units))
dec_lstm_output_attn, state_h_attn, state_c_attn = dec_lstm_attn(dec_embedding_attn, initial_state=decoder_states_inputs_attn)
attn_output = attention([dec_lstm_output_attn, enc_output_input_attn])
dec_combined = Concatenate()([dec_lstm_output_attn, attn_output])
dec_output_attn = Dense(fr_vocab_size, activation='softmax')(dec_combined)
decoder_model_attn = Model([decoder_input_attn, enc_output_input_attn] + decoder_states_inputs_attn,
                          [dec_output_attn, attn_output] + [state_h_attn, state_c_attn])

# Function to translate with attention and return attention weights
def translate_with_attention(input_sentence):
    input_sentence = '<start> ' + re.sub(r'[^\w\s]', '', input_sentence.lower()).strip() + ' <end>'
    input_seq = eng_tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_eng_len, padding='post')

    enc_output, h, c = encoder_model_attn.predict(input_seq, verbose=0)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_tokenizer.word_index['<start>']
    output_tokens = []
    attention_weights = []

    while True:
        output_tokens_probs, attn, h, c = decoder_model_attn.predict([target_seq, enc_output, h, c], verbose=0)
        predicted_token = np.argmax(output_tokens_probs[0, -1, :])
        if predicted_token == fr_tokenizer.word_index['<end>'] or len(output_tokens) > max_fr_len:
            break
        output_tokens.append(predicted_token)
        attention_weights.append(attn[0, -1, :])
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = predicted_token

    output_sentence = ' '.join([fr_tokenizer.index_word.get(token, '') for token in output_tokens])
    return output_sentence, np.array(attention_weights)

# Visualize attention weights
def plot_attention(attn_weights, input_sentence, output_sentence):
    input_tokens = input_sentence.split()
    output_tokens = output_sentence.split()
    plt.figure(figsize=(10, 8))
    plt.imshow(attn_weights, cmap='viridis')
    plt.xlabel('Input Tokens')
    plt.ylabel('Output Tokens')
    plt.xticks(range(len(input_tokens)), input_tokens, rotation=45)
    plt.yticks(range(len(output_tokens)), output_tokens)
    plt.colorbar(label='Attention Weight')
    plt.title('Attention Weights Heatmap')
    plt.tight_layout()
    plt.show()

# Test attention model on two sentences
print("\nTesting attention model:\n")
for sentence in test_sentences[:2]:
    translation, attn_weights = translate_with_attention(sentence)
    print(f"English: {sentence}")
    print(f"French: {translation}\n")
    plot_attention(attn_weights, sentence, translation)


In [ ]:
# Task 7: Plotting Loss and Accuracy
#


# Plot loss and accuracy curves
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss (Basic)')
plt.plot(history.history['val_loss'], label='Val Loss (Basic)')
plt.plot(history_attn.history['loss'], label='Train Loss (Attention)', linestyle='--')
plt.plot(history_attn.history['val_loss'], label='Val Loss (Attention)', linestyle='--')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy (Basic)')
plt.plot(history.history['val_accuracy'], label='Val Accuracy (Basic)')
plt.plot(history_attn.history['accuracy'], label='Train Accuracy (Attention)', linestyle='--')
plt.plot(history_attn.history['val_accuracy'], label='Val Accuracy (Attention)', linestyle='--')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Observations
print("### Task 7 Observations:\n")
print("- **Overfitting**: The training loss decreases steadily, but the validation loss plateaus after a few epochs, indicating "
      "potential overfitting in both models. The attention model shows slightly better validation loss, suggesting improved "
      "generalization.")
print("- **Underfitting**: Early epochs show high loss, but by epoch 15, both models achieve reasonable performance, indicating "
      "no severe underfitting. The attention model converges faster.")
print("- **Training Stability**: Both models show stable training with decreasing loss and increasing accuracy. The attention "
      "model has slightly more stable validation metrics, likely due to better handling of long sequences.")
